In [ ]:
import pickle
import pandas as pd
import numpy as np
from scipy.stats import skew,kurtosis
import gc
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import keras
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Reshape, Flatten, BatchNormalization, LSTM
from keras.optimizers import SGD
import itertools

In [ ]:
data_location = r'C:\Users\atagu\Downloads\data_preprocessed_python\data_preprocessed_python'

In [ ]:
def setup_for_binary():
    for i in range(40):
        if labels['valence'][i] >= 5:
            labels['valence'][i] = 1
        else:
            labels['valence'][i] = 0
        if labels['arousal'][i] >= 5:
            labels['arousal'][i] = 1
        else:
            labels['arousal'][i] = 0

In [ ]:
def prepare_reading_list(sample_no,channel_no,count):
    reading_list = np.array_split(dataset['data'][sample_no][channel_no],count)
    return reading_list

In [ ]:
def get_features_from_reading(data):
    #indivual data for batches
    mean = np.mean(data)
    median = np.median(data)
    max_val = max(data)
    min_val = min(data)
    std = np.std(data)
    variance = np.var(data)
    range_val = max_val - min_val
    skew_val = skew(data)
    kurt_val = kurtosis(data)
      
    
    return_list = []
    return_list.extend([mean,median,max_val,min_val,std,variance,range_val,skew_val,kurt_val])
    return return_list

In [ ]:
def get_features_from_channel(sample_no,channel_no):
    #general data from all 8k
    gen_mean = np.mean(dataset['data'][sample_no][channel_no])
    gen_med = np.median(dataset['data'][sample_no][channel_no])
    gen_max_val = max(dataset['data'][sample_no][channel_no])
    gen_min_val = min(dataset['data'][sample_no][channel_no])
    gen_std = np.std(dataset['data'][sample_no][channel_no])
    gen_variance = np.var(dataset['data'][sample_no][channel_no])
    gen_range_val = gen_max_val - gen_min_val
    gen_skew_val = skew(dataset['data'][sample_no][channel_no])
    gen_kurt_val = kurtosis(dataset['data'][sample_no][channel_no])
    
    return_list = []
    return_list.extend([gen_mean,gen_med,gen_max_val,gen_min_val,gen_std,gen_variance,gen_range_val,gen_skew_val,gen_kurt_val])
    return return_list

In [ ]:
def prepare_data(dataset):
    new_channel_data_list = []
    for i in range(40):
        for k in range(40):
            reading_list = prepare_reading_list(i,k,10)
            temp_list = []
            for j in range(10):
                temp_list.extend(get_features_from_reading(reading_list[j]))
            temp_list.extend(get_features_from_channel(i,k))
            new_channel_data_list.append(temp_list)
    dataset = np.resize(dataset,(40,40,99))
    num = 0
    for i in range(40):
        for k in range(40):
            dataset[i][k] = np.asarray(new_channel_data_list[num])
            num = num+1
    return dataset

In [ ]:
data_list = []
label_list = []

for i in range(1,32):
    if i < 10:
        with open(data_location + '\s0'+ str(i) +'.dat', 'rb') as f: 
            dataset = pickle.load(f, encoding='latin1')
        labels = pd.DataFrame({'valence':dataset['labels'][:,0],'arousal':dataset['labels'][:,1],'dominance':dataset['labels'][:,2],'liking':dataset['labels'][:,3] })
        setup_for_binary()
        labels_enc = pd.get_dummies(labels['valence'])
        label_list.append(labels_enc)
        data_list.append(prepare_data(dataset['data']))
        gc.collect()
    else:
        with open(data_location + '\s'+ str(i)+ '.dat', 'rb') as f: 
            dataset = pickle.load(f, encoding='latin1')
        labels = pd.DataFrame({'valence':dataset['labels'][:,0],'arousal':dataset['labels'][:,1],'dominance':dataset['labels'][:,2],'liking':dataset['labels'][:,3] })
        setup_for_binary()
        labels_enc = pd.get_dummies(labels['valence'])
        label_list.append(labels_enc)
        data_list.append(prepare_data(dataset['data']))
        gc.collect()

In [ ]:
data_list = np.concatenate(data_list)
label_list = np.concatenate(label_list)

In [ ]:
model = Sequential()
model.add(Dense(5000,input_shape=(40,99)))
model.add(Dropout(0.35))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(500))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dense(1000))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2))
model.add(Activation('softmax'))



rmsprop = keras.optimizers.RMSprop(lr=0.00001, rho=0.9, epsilon=None, decay=0.0)
sgd = keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
adagrad = keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer = rmsprop, loss = 'categorical_crossentropy', metrics = ['acc'])
model.fit(data_list,label_list,epochs = 10, batch_size =310, validation_split=0.33, shuffle=True)